In [1]:
# !python --version   
# !pip install langchain --upgrade
# !conda install langchain -c conda-forge --y
# Version: 0.0.149

In [ ]:
# !pip install openai --upgrade
# !conda install openai -c conda-forge --y

In [1]:
OPENAI_API_KEY = ''
OPENAI_API_BASE = 'https://xxx.openai.azure.com'
OPENAI_API_TYPE = 'azure'
OPENAI_API_VERSION = '2022-12-01' # 2023-03-15-preview
PINECONE_API_KEY = ''
PINECONE_API_ENV = 'us-west1-gcp-free'

In [4]:
import openai

openai.api_key = OPENAI_API_KEY
openai.api_base = OPENAI_API_BASE
openai.api_type = OPENAI_API_TYPE
openai.api_version = OPENAI_API_VERSION

# response = openai.Completion.create(
#     engine="chat",
#     prompt="This is a test",
#     max_tokens=5
# )

In [ ]:
# Import Azure OpenAI
# from langchain.llms import AzureOpenAI

In [ ]:
# Replace the deployment name with your own
# llm = AzureOpenAI(
#     deployment_name="text-embedding-ada-002", 
#     model_name="text-embedding-ada-002", 
#     openai_api_key=OPENAI_API_KEY
# )

In [ ]:
# Run the LLM
# llm("Tell me a joke")

In [ ]:
# print(llm)

In [5]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

## Load your data

In [6]:
# loader = UnstructuredPDFLoader("../data/field-guide-to-data-science.pdf")
loader = OnlinePDFLoader("https://github.com/Azure-Samples/azure-search-openai-demo/raw/main/data/Northwind_Standard_Benefits_Details.pdf")

In [ ]:
# !pip install unstructured
# !pip install pdfminer.six

In [7]:
data = loader.load()

In [ ]:
print (f'You have {len(data)} document(s) in your data')
print (f'There are {len(data[0].page_content)} characters in your document')

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [ ]:
print (f'Now you have {len(texts)} documents')

In [ ]:
# !pip install pinecone-client

In [ ]:
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

In [ ]:
embeddings = OpenAIEmbeddings(
    openai_api_key=OPENAI_API_KEY, 
    openai_api_base=OPENAI_API_BASE, 
    openai_api_type=OPENAI_API_TYPE, 
    openai_api_version=OPENAI_API_VERSION,
    chunk_size=1
)

In [ ]:
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "openaidemo" # put in the name of your pinecone index here


In [ ]:
# !pip install tiktoken

In [ ]:
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

In [ ]:
query = "what is included in my health benefit"
docs = docsearch.similarity_search(query, include_metadata=True)

In [ ]:
# Here's an example of the first document that was returned
docs[0].page_content[:250]

In [ ]:
from langchain.llms import OpenAI
# from langchain.llms import AzureOpenAI
from langchain.chains.question_answering import load_qa_chain

In [ ]:
# llm = OpenAI(
#     temperature=0,
#     openai_api_key=OPENAI_API_KEY,
#     openai_api_base=OPENAI_API_BASE,
#     model_name = "text-davinci-003",
#     deployment_id = "davinci",
#     max_tokens=100
# )
llm = AzureOpenAI(
    deployment_name="davinci",
    model_name="text-davinci-003", 
    openai_api_key=OPENAI_API_KEY
)
chain = load_qa_chain(llm, chain_type="stuff")

In [ ]:
# query = "What is the collect stage of data maturity?"
docs = docsearch.similarity_search(query, include_metadata=True)

In [ ]:
print(llm(query))

In [ ]:
chain.run(input_documents=docs, question=query)